In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from sklearn.metrics import accuracy_score
import numpy as np
from category_encoders.target_encoder import TargetEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from catboost.utils import eval_metric
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings('ignore')
import time
from sklearn.model_selection import StratifiedKFold
RANDOM_STATE = 12 
FOLDS = 5

<center>Please do upvote if you find this notebook Useful<center>

In this noteboo we basically do a simple EDA and Apply three models and check each of those accuracy. In those model we have to reduce the loss so we define here those loss on each model.

In [ ]:
#import dataset
train= pd.read_csv('../input/spaceship-titanic/train.csv')
test=pd.read_csv('../input/spaceship-titanic/test.csv')

In [ ]:
train.head(5)

In [ ]:
train.tail(5)

In [ ]:
print("Row of training data", train.shape[0])
print("Column of training data", train.shape[1])

In [ ]:
train.dtypes

In [ ]:
train.columns

In [ ]:
train.nunique()

Below is the basic statistics for each variables which contain information on `count`, `mean`, `standard deviation`, `minimum`, `1st quartile`, `median`, `3rd quartile` and `maximum`.

In [ ]:
#10 columns check
train.describe().T.head(10)

In [ ]:
# drop ID
train = train.drop('PassengerId', axis=1)
test=test.drop('PassengerId', axis=1)

In [ ]:
train.describe().T.drop('count', axis=1).head(10).style.highlight_max(color='darkred')

In [ ]:
train.isnull().sum()

In [ ]:
train["HomePlanet"].value_counts()

In [ ]:
train["CryoSleep"].value_counts()

In [ ]:
train["Cabin"].value_counts()

In [ ]:
# Choose only numerical columns
train.select_dtypes(include=np.number).head()

In [ ]:
train['Transported'].describe()

In [ ]:
#skewness and kurtosis
print("Skewness: %f" % train['Transported'].skew())
print("Kurtosis: %f" % train['Transported'].kurt())

Check relation in between 

In [ ]:
#most correlated 10 columns
corr=train.corr()
v=10
colmn = corr.nlargest(v, 'Transported')
colmn

In [ ]:
fig = px.imshow(train.corr() ,text_auto=True, aspect="auto" , color_continuous_scale = "viridis")
fig.show()

### Pre-Processing

In [ ]:
y=train['Transported']
train = train.drop('Transported', axis=1)

In [ ]:

train.drop(["Name" ,"Cabin"] , axis = 1 ,inplace = True)
test.drop(["Name" ,"Cabin"] , axis = 1 ,inplace = True)

In [ ]:
#Encoding those data
nums_cols = [col for col in train.columns if train[col].dtype in ["float16","float32","float64"]]
catgo_cols = [col for col in train.columns if train[col].dtype not in ["float16","float32","float64"]]

In [ ]:
from sklearn.preprocessing import LabelEncoder
def label_encoder(train,test,columns):
    for col in columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
        train[col] = LabelEncoder().fit_transform(train[col])
        test[col] =  LabelEncoder().fit_transform(test[col])
    return train, test

train ,test = label_encoder(train,test ,catgo_cols)

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(train, 
                                                       y, 
                                                       random_state = 2022,
                                                       test_size =0.23)

In [ ]:
d_test = test

for cols in catgo_cols:
    enc = TargetEncoder(cols=[cols])
    x = enc.fit_transform(X_train, y_train)
    y= enc.transform(X_test)
    z = enc.transform(d_test)

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler= RobustScaler()
scaler.fit(X_train)
train_X = pd.DataFrame(scaler.transform(x))
test_X = pd.DataFrame(scaler.transform(y))
test = pd.DataFrame(scaler.transform(z))

In [ ]:
train_X = train_X.to_numpy()
y_train =y_train.to_numpy()
y_test =y_test.to_numpy()
test_X =test_X.to_numpy()
test =test.to_numpy()

In [ ]:
train_X.shape

In [ ]:
test_X.shape

## Modeling

## XGB

- XGBoost Parameter Tuning: Folow the link for more details. https://www.kaggle.com/prashant111/a-guide-on-xgboost-hyperparameters-tuning/notebook


In [ ]:
from xgboost import XGBClassifier


# declare parameters
params = {
            'objective':'binary:logistic',
#             'objective' : 'multi:softmax',
            'tree_method': 'gpu_hist',
            'eval_metric': 'logloss',
            'booster' : 'gbtree',
            'subsample' : 0.8326,
            'gamma' : 0.48,
            'max_depth': 7,
            'alpha': 10,
            'learning_rate': .027,
            'n_estimators':3000,
            'predictor': 'gpu_predictor'
        }       
           
          
# instantiate the classifier 
xgb = XGBClassifier(**params)
eval_set = [(train_X, y_train), (test_X, y_test)]
xgb.fit(train_X, y_train,
          early_stopping_rounds=200,
#           eval_set=[(test_X,y_test)],
        eval_set=eval_set,
         eval_metric=["error", "logloss"],
          verbose=True)

In [ ]:
y_pred = xgb.predict(test_X)
print('Validation Score:', accuracy_score(y_test, y_pred))

In [ ]:
results = xgb.evals_result()
epochs = len(results['validation_0']['error'])
x_axis = range(0, epochs)

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
ax.legend()

plt.ylabel('Log Loss')
plt.title('XGBoost Log Loss')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
ax.plot(x_axis, results['validation_0']['error'], label='Train')
ax.plot(x_axis, results['validation_1']['error'], label='Test')
ax.legend()

plt.ylabel('Classification Error')
plt.title('XGBoost Classification Error')
plt.show()

## CatBoostClassifier

- CatBoostClassifier Paremeter Tuning follow those link.
- https://www.kaggle.com/saurabhshahane/catboost-hyperparameter-tuning-with-optuna
- https://www.kaggle.com/prashant111/catboost-classifier-in-python
- https://www.kaggle.com/shivampanwar/catboost-and-hyperparameter-tuning-using-bayes

In [ ]:
model = CatBoostClassifier(devices = '0')
eval_set=[(train_X, y_train), (test_X, y_test)]
model.fit(train_X, y_train,  eval_set=eval_set)

### LGBM

- LGBM Paremeter Tuning follow this link 
https://www.kaggle.com/bextuychiev/lgbm-optuna-hyperparameter-tuning-w-understanding

In [ ]:
lgbm = lgb.LGBMClassifier(objective="binary", n_estimators=3000, learning_rate=0.001,seed=8798, random_state=1121218)
eval_set=[(train_X, y_train), (test_X, y_test)]
lgbm.fit(
            train_X,
            y_train,
#             eval_set=[(test_X, y_test)],
            eval_set=eval_set,
            eval_metric="binary_logloss",
            early_stopping_rounds=100,
        )

In [ ]:
lgb.plot_metric(lgbm)

### StratifiedKFold LGBM

In [ ]:
model = lgb.LGBMClassifier(objective="binary", n_estimators=3000, learning_rate=0.001,seed=8798, random_state=1121218)
lgb_predictions = []
lgb_scores = []
lgb_fimp = []

skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)
for fold, (train_idx, valid_idx) in enumerate(skf.split(train_X, y_train)):
    
    print(10*"=", f"Fold={fold+1}", 10*"=")
    start_time = time.time()
    
#     X_train, X_valid = train.iloc[train_idx][train_X], train.iloc[valid_idx][train_X]
#     y_train , y_valid = y_train.iloc[train_idx] , y_train.iloc[valid_idx]
    
    lgbm = model
    lgbm.fit(train_X, y_train,verbose=0)
    
    preds_valid = model.predict(test_X)
    acc = accuracy_score(y_test,  preds_valid)
    lgb_scores.append(acc)
    run_time = time.time() - start_time
    
    print(f"Fold={fold+1}, Accuracy: {acc:.2f}, Run Time: {run_time:.2f}s")
    test_preds = lgbm.predict(test_X)
    lgb_predictions.append(test_preds)
    
print("Mean Accuracy :", np.mean(lgb_scores))

In [ ]:
y_pred = lgbm.predict(test_X)
print('Validation Score:', accuracy_score(y_test, y_pred))

## Submission

#### XGB 

In [ ]:
submission = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')
submission['Transported'] =xgb.predict(test)
submission.to_csv("submission.csv",index=False)
submission.head()

#### LightLGBM

In [ ]:
submission = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')
submission['Transported'] =lgbm.predict(test)
submission.to_csv("submission_3.csv",index=False)
submission.head()

#### CatBoostClassifier

In [ ]:
submission = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')
submission['Transported'] =model.predict(test)
submission.to_csv("submission_2.csv",index=False)
submission.head()